# Detect gait
This script use leave-one-subject-out cross-validation (LOSO-CV) to train a classifier on all but one participant (train set) and predict gait on the participant left out (test set). A classifier is subsequently stored by training on all participants.

Execution time: 

### Modules

In [1]:
from pdathome.classification import train_test_gait_detection, store_gait_detection
from pdathome.constants import global_constants as gc

### Constants

In [2]:
classifier_names = [gc.classifiers.RANDOM_FOREST]

### Process data

In [ ]:
for subject in gc.participant_ids.PD_IDS + gc.participant_ids.HC_IDS:
    train_test_gait_detection(subject, classifier_names=classifier_names)

store_gait_detection(classifier_names, n_jobs=-1)

Storing model output at gait step for classifier rf ...
